In [15]:
import os
import shutil
import pandas as pd

In [16]:
# dataDicts = []
# for root, dirs, files in os.walk('oasis3downloads'):
#     for file in files:
#         if 'nii.gz' in file:
#             filepath = os.path.join(root,file)
#             if '_MR_' in filepath:
#                 type = str(filepath).split('_')[-1]
#                 type = type.split('.nii.gz')[0]
#                 baseFolder = str(filepath).split(os.sep)[1]
#                 newPath = os.path.join('NIFTI',baseFolder,file)
#                 temp = pd.Series()
#                 temp = {'origpath':filepath, 'type':type, 'newpath':newPath}
#                 dataDicts.append(temp)
#             break

In [17]:
df = pd.read_csv('/home/soenke/Internship/DeepDicomSortFork/prototyping/OASIS/extensive2/Oasis3_trainLabels_extensive2.txt',names=['ID','label','extra'],sep='\t',low_memory=False,dtype={'ID':str,'label':int,'extra':int})
df

,ID,label,extra
0,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,17,0
1,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,8,0
2,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,8,0
3,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,17,0
4,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,17,0
...,...,...,...
459945,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,0,0
459946,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,8,0
459947,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,2,0
459948,data/OASIS/OASIS3/train/NIFTI_SLICES/sub-OAS30...,2,0


In [18]:
def extractType(df):
    ID = df['ID']
    ID = ID.split('__s')[0]
    type = ID.split('_')[-1]
    return type

In [19]:
df['type'] = df.apply(extractType,axis=1)

In [20]:
print(df['type'].unique())

['pet' 'T2star' 'fieldmap' 'bold' 'CT' 'T2w' 'dwi' 'T1w' 'FLAIR' 'asl'
 'GRE' 'minIP' 'swi' 'angio' 'pasl' 'e2']


In [21]:
df['description']=df['type']

In [22]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"PD":4,
             "SWI":5, "GRE":6, "T2*":7, "DWI":8, "ADC":9, "BOLD":10,
             "FlowSensitve":11,"PWI":12,"ASL":13,
             "DTI":14,"Other":15}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    dropLabels = ['moco','mpr','hippo','spinal','unknown']
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','other','survey', 'qflow','smartbrain'] #moco is a motion corrected series
    if description in dropLabels:
        return -1
    if description in otherCategories:
        return 15
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 4
    if description == 'T2w':
        return 2
    if 'FLAIR' in description:
        return 3
    if description =='pd':
        return 4
    if description =='swi':
        return 5
    if description.lower() =='gre':
        return 6
    if description =='T2star':
        return 7
    if description =='dwi':
        return 8
    if description =='adc':
        return 9
    if description =='bold':
        return 10
    if description =='angio': #flowsensitive
        return 11
    if description =='pwi' or description=='cbf':
        return 12
    if 'asl' in description:
        return 13
    if description == 'dti':
        return 14
    return -1

In [23]:
df['label'] = df.apply(assignLabel,axis=1)

In [24]:
df['label'].value_counts()

label
15    118925
10     67150
2      55725
0      54000
8      44825
7      31725
13     20800
6      19450
5      18275
3      16900
11     12175
Name: count, dtype: int64

In [25]:
df[df['label']==-1]

,ID,label,extra,type,description


In [26]:
df = df[df['label']!=-1]

In [27]:
df = df[['ID','label','extra']]

In [28]:
df.to_csv('Oasis3_trainLabels_extensive3.txt',index=False, header=False, sep='\t')